Copyright (c) 2023, DKRZ, MPI-M

All rights reserved.

Redistribution and use in source and binary forms, with or without modification,
are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this
   list of conditions and the following disclaimer in the documentation and/or
   other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors
   may be used to endorse or promote products derived from this software without
   specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING,
BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY
OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE
OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
import xarray as xr
import isodate
from datetime import date
from os.path import exists

**ICON/YAC preamble**

In [2]:
VERBOSE = 2

iso_start_date='1979-11-01T00:00:00.000'
iso_end_date='1980-04-01T00:00:00.000'
iso_coupling_interval='P10D'
iso_data_interval='P1M'

dataPath = "/pool/data/ECHAM6/input/r0008/T127/ozone/"
fileRoot = "T127_ozone_"

irad_o3 = 5
lyr_perp = True
yr_perp = 1950

if ( irad_o3 == 6 ) :
    scenario = "picontrol"
if ( irad_o3 == 5 ) :
    scenario = "historical"

# scenario = "ssp119"
# scenario = "ssp126"
# scenario = "ssp370"
# scenario = "ssp585"
# scenario = "perpetual" ; scenario_year = "historical_2014"

**Some more initialisation**

In [3]:
switch_year = False
scenario_year = 2014

start_date = isodate.parse_datetime(iso_start_date)
end_datetime = isodate.parse_datetime(iso_end_date)
coupling_interval = isodate.parse_duration(iso_coupling_interval)
data_interval = isodate.parse_duration(iso_data_interval)

**Reading the data**

In [4]:
def filename_dflt ( dataPath , fileRoot, scenario ) :
    filename =  dataPath+fileRoot+scenario+".nc"
    return filename

def filename_perp ( dataPath , fileRoot, scenario_year ) :
    filename =  dataPath+fileRoot+scenario_year+".nc"
    
    if ( exists(filename) ) :
        return filename
    else :
        print ( filename, " not available!" )
        raise SystemExit(1) ; exit

def filename_year ( dataPath , fileRoot, scenario, year ) :
    if ( scenario[:3] == "ssp" and year > 2014 ) :
        filename = dataPath+fileRoot+scenario+"_"+str(year)+".nc"
    else:
        filename = dataPath+fileRoot+"historical"+"_"+str(year)+".nc"
        
    if ( exists(filename) ) :
        return filename
    else :
        print ( filename, " not available!" )
        raise SystemExit(1); exit

In [5]:
if ( scenario == "picontrol" ) :
    filename = filename_dflt( dataPath, fileRoot ,scenario )
elif ( scenario == "perpetual" ) :
    filename = filename_perp( dataPath, fileRoot, scenario_year )
elif ( scenario[:3] == "ssp" or scenario == "historical" ) :
    filename = filename_year( dataPath, fileRoot, scenario, start_date.year )
else :
    print ( "scenario ", scenario, " not supported!" )
    raise SystemExit(1); exit

if ( VERBOSE > 0 ) :
    print ( "o3_provider: reading ", filename )

dataset = xr.open_dataset(filename, decode_times=False)
other_dataset = dataset

o3_provider: reading  /pool/data/ECHAM6/input/r0008/T127/ozone/T127_ozone_historical_1979.nc


**Time loop**

In [6]:
this_date = start_date
watch_this = 0
watch_other = 0

while this_date <= end_datetime:

    this_month = this_date.month
    this_year = this_date.year

    if ( this_month < 12) :
        mid_of_this_month = (date(this_year, this_month+1, 1) -
                             date(this_year, this_month, 1)).days * 43200
    else :
        mid_of_this_month = 31 * 43200

    sec_of_this_month = (this_date.day-1) * 86400 + \
                         this_date.hour   * 3600 +  \
                         this_date.minute * 60 +    \
                         this_date.second  
    
    if ( sec_of_this_month <= mid_of_this_month ) :
        switch = -1
    else :
        switch = 1

    other_date = this_date + switch * data_interval
    other_month = other_date.month
    other_year = other_date.year

    if ( scenario[:3] == "ssp" or scenario == "historical" ) :

        if ( other_year != this_year and not switch_year ) :
            filename = filename_year( dataPath, fileRoot, scenario, other_year )
            other_dataset = xr.open_dataset(filename, decode_times=False)
            switch_year = True
            if ( VERBOSE > 0 ) :
                print ( "o3_provider: Opened dataset for year ", other_year )

        if ( other_year == this_year and switch_year ) :
            switch_year = False
            other_dataset = dataset
            if ( VERBOSE > 0) :
                print ( "o3_provider: Complete switch to year ", other_year )

    if ( other_month < 12) :
        mid_of_other_month = (date(this_year, other_month+1, 1) -
                              date(this_year, other_month, 1)).days * 43200
    else :
        mid_of_other_month = 31 * 43200
 
    other_wght = switch * ( sec_of_this_month - mid_of_this_month ) / \
                          ( mid_of_this_month + mid_of_other_month )
    this_wght  = 1 - other_wght

    if ( this_month != watch_this ) :
        if ( watch_this == 0 ) :
            if ( VERBOSE > 1 ) : print ( "o3_provider: Reading this month", this_month )
            o3_this_month  = dataset["O3"][{"time":this_month-1}].values.reshape((1,-1))

        else :
            if ( VERBOSE > 1 ) : print ( "o3_provider: Flipp this month to ", this_month )
            o3_this_month = o3_other_month

        watch_this = this_month

    # with an extra copy one could save some more reading from file
    if ( other_month != watch_other ) :
        if ( VERBOSE > 1 ) : print ( "o3_provider: Reading other month", other_month )
        o3_other_month = other_dataset["O3"][{"time":other_month-1}].values.reshape((1,-1))
        watch_other = other_month

    if ( VERBOSE > 1 ) :
        print ("o3_provider: ", this_date.isoformat() , ' wght ' , 
               "%8.6f" % this_wght, '*' , "%2i" % this_month , '+' ,
               "%8.6f" % other_wght, '*', "%2i" % other_month)

    o3_array = this_wght * o3_this_month + other_wght * o3_other_month

    this_date = this_date + coupling_interval

print ( "The End" )

o3_provider: Reading this month 11
o3_provider: Reading other month 10
o3_provider:  1979-11-01T00:00:00  wght  0.508197 * 11 + 0.491803 * 10
o3_provider:  1979-11-11T00:00:00  wght  0.836066 * 11 + 0.163934 * 10
o3_provider: Reading other month 12
o3_provider:  1979-11-21T00:00:00  wght  0.836066 * 11 + 0.163934 * 12
o3_provider: Flipp this month to  12
o3_provider: Reading other month 11
o3_provider:  1979-12-01T00:00:00  wght  0.491803 * 12 + 0.508197 * 11
o3_provider:  1979-12-11T00:00:00  wght  0.819672 * 12 + 0.180328 * 11
o3_provider: Opened dataset for year  1980
o3_provider: Reading other month 1
o3_provider:  1979-12-21T00:00:00  wght  0.854839 * 12 + 0.145161 *  1
o3_provider:  1979-12-31T00:00:00  wght  0.532258 * 12 + 0.467742 *  1
o3_provider: Flipp this month to  1
o3_provider: Reading other month 12
o3_provider:  1980-01-10T00:00:00  wght  0.790323 *  1 + 0.209677 * 12
o3_provider: Complete switch to year  1980
o3_provider: Reading other month 2
o3_provider:  1980-01-20